In [27]:
import json 
import pickle
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelEncoder
import colorama 
import random
from colorama import Fore, Style, Back


In [9]:
with open('file.json') as file:
 data = json.load(file)
 
training_sentences = []

training_labels = []
labels = []
responses = []
for intent in data['intents']:
 for pattern in intent['patterns']:
  training_sentences.append(pattern)
  training_labels.append(intent['tag'])
  responses.append(intent['responses'])
 
 if intent['tag'] not in labels:
   labels.append(intent['tag'])
 
num_classes = len(labels)
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [11]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', 
maxlen=max_len)


In [13]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', 
 optimizer='adam', metrics=['accuracy'])
model.summary()
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), 
epochs=epochs)



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (  (None, 16)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 12)                204       
                                                                 
Total params: 16748 (65.42 KB)
Trainable params: 16748 (65.42 KB)
Non-trainable params: 0 (0.00 Byte)
__________________

In [35]:
model.save("chat_model")

with open('tokenizer.pickle', 'wb') as handle:
 pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
 
with open('label_encoder.pickle', 'wb') as ecn_file:
 pickle.dump(lbl_encoder, ecn_file, 
protocol=pickle.HIGHEST_PROTOCOL)


INFO:tensorflow:Assets written to: chat_model\assets


INFO:tensorflow:Assets written to: chat_model\assets


In [ ]:
def chat():
    # Load trained model
    model = keras.models.load_model('chat_model')
    
    # Load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)
    
    # Load intents file
    with open("file.json") as file:
        data = json.load(file)

    print("Bot: Hello! How can I assist you today?")
    
    while True:
        # Take user input
        user_input = input("You: ")
        
        # Exit condition
        if user_input.lower() in ["bye", "exit", "quit"]:
            print("Bot: Goodbye!")
            break
        
        # Tokenize user input
        input_tokens = tokenizer.texts_to_sequences([user_input])
        input_tokens = pad_sequences(input_tokens, padding='post', maxlen=20)  # Adjust maxlen as per model's training
        
        # Get model's prediction
        prediction = model.predict(input_tokens)
        intent_idx = np.argmax(prediction)  # Get index of the highest predicted probability
        
        # Retrieve the corresponding intent response
        intent = data['intents'][intent_idx]
        response = random.choice(intent['responses'])
        
        print(f"Bot: {response}")

if __name__ == "__main__":
    chat()



Bot: Hello! How can I assist you today?


You:  hello


1/1 [==============================] - 0s 152ms/step
Bot: Trending books this week: 'Lessons in Chemistry,' 'The 48 Laws of Power,' and 'It Ends With Us.'


You:  i want a ook


1/1 [==============================] - 0s 35ms/step
Bot: You can find amazing discounts in our Promotions section. Donâ€™t miss out!


You:  physics books


1/1 [==============================] - 0s 24ms/step
Bot: Weâ€™ve got a wide range: fiction, non-fiction, fantasy, mystery, romance, sci-fi, and more!
